In [2]:
import findspark
findspark.init()

import datetime
import requests
from pyspark.sql import SparkSession


spark = (SparkSession.builder
      .master("local[1]")
      .appName("ChallengeDataside.com")
      .getOrCreate())

requisicao = requests.get("https://servicodados.ibge.gov.br/api/v1/localidades/mesorregioes/3513/municipios")
dfCities = requisicao.json()
data = dfCities
df = spark.createDataFrame(data)
df.createOrReplaceTempView("Cities")


API_KEY = "*******************************"

municipio = "Aparecida"
link = f"https://api.openweathermap.org/data/2.5/forecast?q={municipio}&appid={API_KEY}&lang=pt_br"
requisicaoApi = requests.get(link)
requisicaoApiDic = requisicaoApi.json()
df = requisicaoApiDic

cidade = requisicaoApiDic['city']['name']
codigoCidade = requisicaoApiDic['city']['id']
dataPrevisao = requisicaoApiDic['list'][0]['dt_txt']
regiao = ""
pais = requisicaoApiDic['city']['country']
lat = requisicaoApiDic['city']['coord']['lat']
long = requisicaoApiDic['city']['coord']['lon']
tempMax = requisicaoApiDic['list'][0]['main']['temp_max'] - 273.15
tempMin = requisicaoApiDic['list'][0]['main']['temp_min'] - 273.15
tempMed = tempMax + tempMin / 2
requisicaoApiDic['list'][0]['weather'][0]['main'] = requisicaoApiDic['list'][0]['weather'][0]['main'].replace("Rain", "Sim")
requisicaoApiDic['list'][0]['weather'][0]['main'] = requisicaoApiDic['list'][0]['weather'][0]['main'].replace("Clear", "Não")
requisicaoApiDic['list'][0]['weather'][0]['main'] = requisicaoApiDic['list'][0]['weather'][0]['main'].replace("Clouds", "Não")
vaiChover = requisicaoApiDic['list'][0]['weather'][0]['main']
condicaoDoTempo = requisicaoApiDic['list'][0]['weather'][0]['description']
nascerDoSol = requisicaoApiDic['city']['sunrise']
nascerDoSol = datetime.datetime.fromtimestamp(nascerDoSol).strftime('%H:%M:%S')
porDoSOl = requisicaoApiDic['city']['sunset']
porDoSOl = datetime.datetime.fromtimestamp(porDoSOl).strftime('%H:%M:%S')
velocidadeDoVento = requisicaoApiDic['list'][0]['wind']['speed']




Commit Repository
